# MAIC Q-Learning Workshop

Lets get the boring stuff out of the way first

We're going to use a couple terms a lot so let's define them here as a reference
- State: The current environment, in our example for tic tac toe, it will be how x's and o's are currently arranged on the board
- Action: A way to represent something we can do. In this example we'll have one action for every space on the board to represent marking it with an x or o
- Reward: For executing a specific <strong>action</strong> on a specific <strong>state</strong> we get rewarded (or punished) based on the result (if we win the game, get a reward!)
- Episode: We'll need to have our program play LOTS of tic tac toe games, an episode represents a single game
- Q-function: This is a function that takes in a <strong>state</strong> and an <strong>action</strong> as inputs, and outputs a quantification of how "good" taking that action is
    - Note that while I'm calling this a function, it's actually going to be implemented as a table, the table will start all inputs at zero and slowly learn what to map them to

## The Bellman Equation
This will be the key part of our algorithm. We don't want our program to take whatever the best immediate action is, we want it to learn to look multiple steps ahead, The Bellman Equation will help us with this. 
$$Q(S,A)=Q(S,A)+\alpha[R(S,A)+\lambda max(Q'(S',A'))-Q(S,A)]$$
First we can see that Q(S,A) is updated with itself as the first part of it's equation, so our algorithm is going to need to be iterative. ($Q(S,A)=Q(S,A)$). <br/>The next section of our equation starts with $\alpha$ This is our <i>learning rate</i>, it's just meant to slow the model down a little, I haven't really found that it makes too big of an impact.<br/>The next portion of our function is $R(S,A)$ which is just the reward for our current state and action. We then add to it $\lambda max(Q'(S',A'))$, $\lambda$ is called the <i>discount</i>, which is meant to decay the future reward, so rewards that are closer to us are worth more (would you fly to Kansas for a pizza or go to ians? Most likely ians because it's closer even though the result is the same). Then we look at all of the possible rewards that can be gained from the next state. The ' (prime) means "next".<br/>Finally we subtract the reward of our current state, this makes a measure of how "important" our current state is

One last thing worth noting is the <strong>Epsilon Greedy Policy</strong><br/>We'll again have some quick definitions
- exploration: gathering new information, ie, rewards for actions we haven't taken for states yet, or looking back at an old state
- exploitation: using our current knowledge to make what is believed to be the best move<br/>

The TLDR is that we want our model to try new moves in order to get a better understanding of the game, so we have some epsilon e, which we decay while training, we'll have an epsilon percent chance on each step to make a random move to ensure we still are exploring new areas

To help us focus on just the Q-Learning part of the code, the tic tac toe implimentation as well as the GUI have been seperated out into a different file.

In [ ]:
# We start with imports as per usual
from tictactoe import TicTacToeEnv
from tqdm import tqdm
import numpy as np
import time

# This isn't too important, it just generates all 9 possible moves in a format expected from our environment
actions = np.zeros((9,3,3))
action_idx = 0
for i in range(3):
    for j in range(3):
        actions[action_idx,i,j] = 1
        action_idx+=1

The code can be confusing so let's look at some psudocode on what the core part of the algorithm will be.<br/><br/>
Initialize $Q(S, A)$ arbitrarily<br/>
for each episode:<br/>
&nbsp;&nbsp;&nbsp;&nbsp;    initialize $S$<br/>
&nbsp;&nbsp;&nbsp;&nbsp;    while $S$ is not terminal:<br/>
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;        choose $A$ from $S$ using a policy from Q<br/>
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;           (epsilon greedy says use q-table or random value)<br/>
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;        take action $A$ and observe $R, S'$<br/>
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;        update $Q(S,A)=Q(S,A)+\alpha[R(S,A)+\lambda max(Q'(S', A'))-Q(S,A)]$<br/>
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;(the Bellman Equation from before)<br/>
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;       update $S$ = $S'$<br/>

In [ ]:
# These are the hyperparameters we talked about earlier
num_episodes = 10000 # We'll play 10000 games of tic tac toe to train
# The next two variables could probably be anywhere from 0.5 to 0.999 and still work just fine, test it out
alpha = 0.9 # our learning rate
gamma = 0.8 # our discoutn factor
# The next three will determine how quickly epsilon will decay, or how quickly we stop exploring
epsilon = 1
epsilon_min = 0.01
epsilon_decay = 0.01
# "space" is used to represent how many of something we have, so we have 9 possible actions
action_space = 9
# This is an empty dictionary that we will use to store the q-values of states and their corresponding actions
q_table = {}
# We want to play as X (go first) so we can set this to 1, it should work too if set to 2
model_turn = 1
# evaluation tracking
wins = 0

# We can set model_turn to 2 to play as o instead of x
# render=True will create a gui for the game
# wait=x will make the game wait x seconds between turns, I reccomend leaving render and wait off during training
env = TicTacToeEnv(player=model_turn)

In [ ]:
# This is our main training loop
for i in tqdm(range(num_episodes)):
    # print out some stats throughout our training
    if i > 0 and i % (num_episodes/10) == 0:
        print(f'win rate: {int(100*wins/i)}%')

    # reset to a new game at the start of each episode
    state, _, _, _ = env.reset()

    # Loop though a game
    while True:
        # If the board isn't already in our q table, add it and set the reward of all actions from that board to zero
        if state.tobytes() not in q_table.keys():
            q_table[state.tobytes()] = np.zeros(action_space)

        # Figure out all of the usable actions for this board so we can punish trying to make bad moves, and not let our epsilon policy try to chose a non possible move
        usable_actions = []
        for i, action in enumerate(actions):
            if env.check_action_usable(action):
                usable_actions.append(i)
    
        # Using epsilon greedy (to ensure exploration) we make a random choice sometimes for one of our usable actions
        if np.random.random(1) < epsilon:
            action_idx = np.random.choice(usable_actions)
        else:
            # or use our model to get the action with the maximum reward
            action_idxs = np.argsort(-q_table[state.tobytes()])
            for i in action_idxs:
                if i in usable_actions:
                    action_idx = i
                    break
                else:
                    q_table[state.tobytes()][i] = -1

        # We can pass in our selected action to env.step() which will execute the current action and return the resulting state, reward, and if the game is over or not
        next_state, reward, done, _ = env.step(actions[action_idx])
        
        # again ensure the board is in our q-table
        if next_state.tobytes() not in q_table.keys():
            q_table[next_state.tobytes()] = np.zeros(action_space)

        # Bellman Equation woohoo
        q_value = q_table[state.tobytes()][action_idx]
        max_next_q = max(q_table[next_state.tobytes()])
        q_table[state.tobytes()][action_idx] = q_value + alpha * (reward + gamma * max_next_q - q_value)

        # don't forget this part
        state = next_state

        # if we done, track the win and break out of the game loop
        if done:
            if reward == 1:
                wins += 1
            break

        # decay epsilon
        epsilon = max(epsilon_min, epsilon-epsilon_decay)
env.close()

In [ ]:
# Inference
# If the gui isn't showing up, check that it isn't behind your IDE
inference_episodes= 5
env = TicTacToeEnv(player=model_turn, render=True, wait=1)

for i in tqdm(range(inference_episodes)):
    # reset to a new game at the start of each episode
    state, _, _, _ = env.reset()

    # Loop though a game
    while True:
        # If the board isn't already in our q table, add it and set the reward of all actions from that board to zero
        if state.tobytes() not in q_table.keys():
            q_table[state.tobytes()] = np.zeros(action_space)

        # Figure out all of the usable actions for this board so we can punish trying to make bad moves, and not let our epsilon policy try to chose a non possible move
        usable_actions = []
        for i, action in enumerate(actions):
            if env.check_action_usable(action):
                usable_actions.append(i)
    
        # or use our model to get the action with the maximum reward
        action_idxs = np.argsort(-q_table[state.tobytes()])
        for i in action_idxs:
            if i in usable_actions:
                action_idx = i
                break

        # We can pass in our selected action to env.step() which will execute the current action and return the resulting state, reward, and if the game is over or not
        next_state, reward, done, _ = env.step(actions[action_idx])

        # don't forget this part
        state = next_state

        # if we done, track the win and break out of the game loop
        if done:
            time.sleep(1)
            break
env.close()

# Challenge Time
## Easy (1pt)
- Our model does pretty good right now, but we're currently only rewarding our model for winning. We should punish it for losing or drawing, open up tictactoe.py and find the step() function definition, try changing the value returned for draws and losses to -1 and -2 respectively. Does the model train faster or better? 
## Medium (2pt)
- Try training with model_turn set to 2. This will start off much worse because going first in tic tac toe makes the game much easier. Try to get similar win rates as having model_turn set to 1. Show the Eboard how you accomplished this
## Hard (3pt)
- What if we tried training TWO models, one for x and one for o. Then what if we pit those models against each other? The current step() implimentation uses a random choice for the "opponent", but try doing iterations of training where you train two models seperately and use one to train the other. 
- Q-Learning can break down with an opponent that also "learns" but what we want is an agent that learns to play against a bad opponent first, then against a better opponent, so on and so forth.
## Hard (3pt)
- Use Q-Learning to solve another game (hint, use google and the words "openai gym", our implimentation should fit well with one of their environments, maybe even the Frozen Lake game)
## A Little More Hard (4pt)
- Tic Tac Toe is a solved game, make a perfect AI for each player. They should be able to draw every time, and x should win or draw every time when not faced with a perfect o AI